Connect Robot, Thymio Suite must be open and thymio must be connected to pc!

In [1]:
import math
import time
import ast
import openai
import tdmclient.notebook
import tdmclient.tools

In [2]:
await tdmclient.notebook.start()

Thymio Program

In [3]:
%%run_python

state = 0


@onevent
def move(distance):
    global motor_left_target, motor_right_target, timer_period
    if distance > 0:
        timer_period[0] = 160 * distance
        motor_left_target = 200
        motor_right_target = 200
    if distance < 0:
        timer_period[0] = 160 * -distance
        motor_left_target = -200
        motor_right_target = -200
    if distance == 0:
        timer_period[0] = 100
    emit("motion", 1)

@onevent
def rotate(angle):
    global motor_left_target, motor_right_target, timer_period
    if angle > 0:
        timer_period[0] = angle * 26
        motor_left_target = -100
        motor_right_target = 100
    if angle < 0:
        timer_period[0] = -angle * 26
        motor_left_target = 100
        motor_right_target = -100
    if angle == 0:
        timer_period[0] = 100
    emit("motion", 1)

""" @onevent
def prox():
    global motor_left_target, motor_right_target, prox_horizontal, state
    if state == 0:
        for i in range(5):
            if prox_horizontal[i] > 2000:
                state = 1
                timer_period[0] = 1000
                motor_left_target = 0
                motor_right_target = 0
                emit("problem", 0)
    if state == 3:
        motor_right_target = (prox_horizontal[0] + prox_horizontal[1]) // 20
        motor_left_target = (prox_horizontal[3] + prox_horizontal[4]) // 20 """


@onevent
def turn(right_speed, left_speed, timer):
    global motor_left_target, motor_right_target, timer_period
    motor_left_target = left_speed
    motor_right_target = right_speed
    if timer == 0:
        timer_period[0] = 100
    else:
        timer_period[0] = timer
    emit("motion", 1)


@onevent
def follow(timer):
    global state
    state = 3
    timer_period[0] = timer

@onevent
def stopfollow(_):
    global state
    state = 0
    timer_period[0] = 5

@onevent
def timer0():
    global motor_left_target, motor_right_target
    if state == 0:
        timer_period[0] = 0
        motor_left_target = 0
        motor_right_target = 0
        emit("done", 0)
    if state == 1:
        state = 0
    



Logic to run on computer

In [52]:
import math

def calculate_turn(radius, angle):
    wheel_distance = 10.1
    velocity = 5.8
    
    radian_angle = math.radians(angle)  # Convert angle from degrees to radians
    arc_length = radius * abs(radian_angle)  # Calculate the arc length the robot needs to travel

    # Calculate the target velocities for each wheel
    if angle == 0:
        motor_left_target = motor_right_target = velocity
    else:
        motor_right_target = velocity * (1 + wheel_distance / (2 * radius)) if angle > 0 else velocity * (1 - wheel_distance / (2 * radius))
        motor_left_target = velocity * (1 - wheel_distance / (2 * radius)) if angle > 0 else velocity * (1 + wheel_distance / (2 * radius))

    return int(motor_left_target * 100/3.2), int(motor_right_target * 100/3.2), int((arc_length / velocity) * 1000)


def get_expected_time(event_name, args):
    match event_name:
        case "move":
            return 0.16 * args[0]
        case "rotate":
            return 0.026 * args[0]
            

Thymio related functions

Function based var updates

In [5]:
@tdmclient.notebook.sync_var
def update_speed(left, right):
    motor_left_target = left
    motor_right_target = right

@tdmclient.notebook.sync_var
def get_current_speed():
    return motor_left_target, motor_right_target

@tdmclient.notebook.sync_var
def get_prox():
    return prox_horizontal

@tdmclient.notebook.sync_var
def set_timer(time, i=0):
    timer_period[i] = time * 1000

Event based

In [54]:
import math, signal

class Done(Exception):
    pass

class Problem(Exception):
    pass

class EventTimeout(Exception):
    pass


# In case the robot fails to receive an event (happens alot :( ) raise this expection
def timeout_handler(sig, NULL):
    raise EventTimeout

def parse_input(task_str):
    actions = ast.literal_eval(task_str)
    return actions

def pop_and_send(event_list, past_events) -> bool:
    if len(event_list) > 0:
        event = event_list.pop(0)
        past_events.append(event)
        event_name = event[0]
        match event_name:
            case "turn":
                right_speed, left_speed, timer = calculate_turn(event[1], event[2])
                expected_time = timer / 1000
                print(right_speed, left_speed, timer)
                send_event("turn", right_speed, left_speed, timer)
                print("Sent event")
            case _:
                expected_time = get_expected_time(event[0], event[1:])
                send_event(event[0], *event[1:])
                print("Sent event")
        # Set an alarm with the aproximate time the robot will take to complete
        # the action plus a 1 second delay
        signal.alarm(int(expected_time) + 1.5)
        return True
    else:
        return False

def execute_actions(actions): 
    # Define handler for the alarm
    signal.signal(signal.SIGALRM, timeout_handler)
    event_list = actions.copy()
    past_events = []

    def on_event_data(node, event_name):
        print(f"Receiving event data for event {event_name}")
        data = get_event_data(event_name)
        print(data)
        if event_name == "done":
            clear_event_data(event_name)
            success = pop_and_send(event_list, past_events)
            if not success:
                signal.alarm(0)  # Cancel the alarm
                raise Done
        elif event_name == "problem":
            signal.alarm(0)
            clear_event_data(event_name)
            raise Problem
        elif event_name == "motion":
            clear_event_data(event_name)
        else:
            raise Exception("Unknown event")

    pop_and_send(event_list, past_events) 
    try:
        tdmclient.notebook.process_events(all_nodes=False, on_event_data=on_event_data)
    except Done:
        print("Robot task completed successfully")
        return "done", None
    except Problem:
        print("Robot encounted a problem")
        return "problem", event_list
    except EventTimeout:
        print("Missed event, resending")
        event_list.append(past_events[-1])
        execute_actions(event_list)


@tdmclient.notebook.sync_var
def main(response=None):
    quit = False
    if response is None:
        tasks = [parse_input(input("Please input a list of actions for the robot to perform"))]
    else:
        tasks = [parse_input(response)]
    while not quit:
        if len(tasks) > 0:
            response, left_over_events = execute_actions(tasks.pop())
            match response:
                case "done":
                    if len(tasks) == 0:
                        quit = True
                case "problem":
                    if left_over_events is not None:
                        tasks.append(left_over_events)
                    tasks.append(parse_input(input("Robot encoutered a problem (*Insert short description*).\nPlease provice extra guidance.")))
            

Unchecked Raise(exc=Name(id='EventTimeout', ctx=Load()))
Unchecked MatchAs()


In [ ]:
@tdmclient.notebook.sync_var
def get_acc():
    print(acc)

while True:
    get_acc()
    time.sleep(.1)

In [53]:
main()

89 272 5416
Sent event
Missed event, resending
89 272 5416
Sent event
Receiving event data for event motion
[[1]]
Receiving event data for event done
[[0]]
Robot task completed successfully


TypeError: cannot unpack non-iterable NoneType object

ChatGPT integration

In [55]:
with open("apikey", "r") as file:
    key = file.read()

openai.api_key = key

MODEL_GTP3_5 = "gpt-3.5-turbo"
MODEL_CURIE = "curie"
response = openai.ChatCompletion.create(
    model=MODEL_GTP3_5,
    messages=[
        {"role": "user", "content": 
         """Imagine you are a two wheeled robot with a wheelabase of 10 cm that can move in a XY plane,
          you move through a differential drive system. The main functions you can use
          to move arround are:
          move(x: int): Moves the robot linearly either x cm to the front or x cm backwards.
          rotate(angle: int): Rotates the robot by a number of degrees without changing the XY position.
          turn(radius where radius > 5, angle): Performs a turn with the given radius and with the correspoding arc to the angle. Radius cannot be negative and must greater than 5.
          
          Can you give a list of actions for the robot to perform for example:
          [("turn",10,90),("rotate",45),("move",-10)]

          Only respond in these lists

          Here is your task:
          
          """  + input(),
          }
    ]
)

print("GPT-3.5 Response:")
print(response.choices[0].message.content)
main(response.choices[0].message.content)


GPT-3.5 Response:
and you cannot turn sharply. Start at (0,0) facing towards the positive X direction.

          [("move", 5), ("turn", 5, 90), ("move", 5), ("turn", 5, -90), ("move", -5), 
           ("rotate", -90), ("move", 5), ("turn", 5, 90), ("move", 5), ("turn", 5, -90), 
           ("move", -5), ("rotate", -90), ("move", 5)]


SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
calculate_turn(7, 45)

(59, 355, 947)